# Trabalho 1 - Recolha, Análise e Tokenização de um Corpus
## Processamento Computacional da Língua

André Eusébio (127600)

Filipe Pereira (131531)

In [22]:
!python3 -m spacy download pt_core_news_lg --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.1 MB/s eta 0:00:0000:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [23]:
import requests
from bs4 import BeautifulSoup
import re
import random
import math
import requests
import spacy
from collections import Counter, defaultdict
from spacy.lang.pt.stop_words import STOP_WORDS
from collections import defaultdict

spacy.prefer_gpu() #usar o gpu

False

## Recolha do Corpus

#### Webscraping Code (usar o .txt file em vez de fazer tudo de novo)

In [24]:
list_urls = [
    'https://blasfemias.net/2025/02/19/tentacao-intima-gigantesca/',
    'https://blasfemias.net/2025/02/03/o-malas/',
    'https://blasfemias.net/2025/02/12/disparates-sobre-comercio-internacional/',
    'https://blasfemias.net/2025/02/25/o-parlamento-sempre-foi-mal-frequentado/',
    'https://www.pensador.com/poemas_curtos_de_fernando_pessoa/',
    'https://www.pensador.com/os_melhores_poemas_de_fernando_pessoa/',
    'https://mag.sapo.pt/showbiz/artigos/leitoes-deixados-a-morrer-a-fome-numa-polemica-exposicao-de-arte-na-dinamarca?utm_source=SAPO_HP&utm_medium=web&utm_campaign=destaques',
    'https://tek.sapo.pt/noticias/ciencia/artigos/james-webb-revela-atmosfera-complexa-de-super-jupiter-sem-estrela?utm_source=SAPO_HP&utm_medium=web&utm_campaign=destaques',
    'https://tek.sapo.pt/expert/artigos/universidade-de-coimbra-chega-a-final-de-competicao-mundial-de-computacao-quantica?utm_source=SAPO_HP&utm_medium=web&utm_campaign=destaques',
    'https://www.jn.pt/5830829273/trump-recebeu-carta-importante-de-zelensky-sobre-acordo-de-minerais/',
    'https://www.reddit.com/r/portugal/comments/1ivkmji/bitoque_que_comi_num_restaurante_portugu%C3%AAs_em/',
    'https://www.reddit.com/r/portugal/comments/1ik95ms/o_tuga_%C3%A9_mesmo_um_animal/',
    'https://www.reddit.com/r/portugal/comments/1ip67rt/expresso_a_usar_imagem_gerada_por_ia_na_capa/',

]

random.shuffle(list_urls)

text = str()

headers = {'User-Agent': 'Mozilla/5.0'}

for url in list_urls:
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.select_one('h1')
        paragraphs = soup.find_all('p')
        span = soup.select_one('span')

        text += ' ' + (title.text if title else '')
        for p in paragraphs:
            text += ' ' + p.text
        text += ' ' + (span.text if span else '')

    except Exception as e:
        print(f'Error: {url} - {e}')

text = re.sub(r'\s+', ' ', text).strip()
text = re.sub(r'(\[\d+\])+', '', text)  # Remove referências [1][2] da wikipedia

print(text)

with open("scraped.txt", "w", encoding="utf-8") as file:
    file.write(text)

Error: https://www.pensador.com/poemas_curtos_de_fernando_pessoa/ - 404 Client Error: Not Found for url: https://www.pensador.com/poemas_curtos_de_fernando_pessoa/
Bitoque que comi num restaurante português em Osaka, Japão Reddit and its partners use cookies and similar technologies to provide you with a better experience. By accepting all cookies, you agree to our use of cookies to deliver and maintain our services and site, improve the quality of Reddit, personalize Reddit content and advertising, and measure the effectiveness of advertising. By rejecting non-essential cookies, Reddit may still use certain cookies to ensure the proper functionality of our platform. For more information, please see our Cookie Notice and our Privacy Policy. Bacalhau! Um espaço para partilhar e discutir Portugal ou portugueses no mundo. Estava delicioso. O que acham? Opiniões? Create your account and connect with a world of communities. Bacalhau! Anyone can view, post, and comment to this community Skip

#### .txt file

In [25]:
with open("scraped.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Contar as palavras
wordCount = len(text.split())

print(f'numero de palavras: {wordCount}')

numero de palavras: 12227


In [26]:
def split_text(text):
    """
    Split the input text into a training and testing set.

    Parameters:
    text: The input text to be split (str)

    Returns:
      A tuple containing the training and testing sets (tuple)
    """

    split_index = math.floor(len(text) * 0.9)
    train_text = text[:split_index]
    test_text = text[split_index:]
    return train_text, test_text

train, test = split_text(text)

print("Train:", train)
print("Test:", test)

Train: Bitoque que comi num restaurante português em Osaka, Japão Reddit and its partners use cookies and similar technologies to provide you with a better experience. By accepting all cookies, you agree to our use of cookies to deliver and maintain our services and site, improve the quality of Reddit, personalize Reddit content and advertising, and measure the effectiveness of advertising. By rejecting non-essential cookies, Reddit may still use certain cookies to ensure the proper functionality of our platform. For more information, please see our Cookie Notice and our Privacy Policy. Bacalhau! Um espaço para partilhar e discutir Portugal ou portugueses no mundo. Estava delicioso. O que acham? Opiniões? Create your account and connect with a world of communities. Bacalhau! Anyone can view, post, and comment to this community Skip to main content James Webb revela atmosfera complexa de “super Júpiter” sem estrela SIMP 0136 é um objeto único na Via Láctea. Localizado a apenas 20 anos-l

## Documentação e caracterização do corpus

### spaCy

In [27]:
# Carregar o modelo em português
nlp = spacy.load("pt_core_news_lg")

# Processar o texto
doc = nlp(text)

# Contar tokens e tipos únicos (riqueza lexical)

# Opção 1 Com todas as palavras
tokens = [token.text.lower() for token in doc if token.is_alpha]
num_palavras = len(tokens)
num_palavras_unicas = len(set(tokens))
riqueza_lexical = num_palavras_unicas / num_palavras if num_palavras > 0 else 0

# Opção 2 Sem stopwords
# Usando a lista de stopwords explícita do spaCy para garantir funcionamento
tokens_sem_stopwords = [token.text.lower() for token in doc if token.is_alpha and token.text.lower() not in STOP_WORDS]
num_palavras_sem_stop = len(tokens_sem_stopwords)
num_palavras_unicas_sem_stop = len(set(tokens_sem_stopwords))
riqueza_lexical_sem_stop = num_palavras_unicas_sem_stop / num_palavras_sem_stop if num_palavras_sem_stop > 0 else 0

# Frequência das palavras mais comuns (com stopwords)
freq = Counter(tokens)
palavras_mais_comuns = freq.most_common(10)

# Frequência das palavras mais comuns (sem stopwords)
freq_s = Counter(tokens_sem_stopwords)
palavras_mais_comuns_s = freq_s.most_common(10)  # Corrigido: estava usando freq em vez de freq_s

# Neologismos (palavras fora do vocabulário)
neologismos = [token.text for token in doc if token.is_alpha and token.is_oov]

# Estrangeirismos
# Considerar palavras com letras pouco comuns em português (w, k) ou combinações incomuns
estrangeirismos = []
for token in doc:
    if token.is_alpha and (
        'w' in token.text.lower() or
        'k' in token.text.lower() or
        'y' in token.text.lower() or
        token.text.lower().endswith('ing')
    ):
        estrangeirismos.append(token.text)


# Lista de gírias comuns em português
top_girias = [
    "fixe", "porreiro", "bué", "giro", "brutal", "pá", "malta", "puto", "gajo", "gaja",
    "bica", "desenrascar", "bora", "bazar", "ralar", "pimba", "bombar", "pancada", "pegar", "véio",
    "pica", "putos", "bicha", "piropo", "sandes", "talho", "pastilha", "sumo", "chávena", "treta",
    "tuga", "mano", "Cueca", "Rabo", 'bro',
    "Propina", "Paragem", "Canalha"
]


# Deteção de gírias
girias = []
for token in doc:
    if token.text.lower() in top_girias:
        girias.append(token.text)

# Mostrar algumas stopwords para verificação
print(f"Exemplos de stopwords em português: {list(STOP_WORDS)[:10]}")

# Exibir resultados
print(f"Número total de palavras: {num_palavras}")
print(f"Número total de palavras únicas: {num_palavras_unicas}")
print(f"Riqueza lexical: {riqueza_lexical:.4f}")
print(f"Palavras mais comuns: {palavras_mais_comuns}")
print(200*'-')
print(f"Número de palavras sem stopwords: {num_palavras_sem_stop}")
print(f"Número de palavras únicas sem stopwords: {num_palavras_unicas_sem_stop}")
print(f"Riqueza lexical sem stopwords: {riqueza_lexical_sem_stop:.4f}")
print(f"Palavras mais comuns (sem stopwords): {palavras_mais_comuns_s}")
print(200*'-')
print(f"Neologismos encontrados: {set(neologismos)}")
print(f"Estrangeirismos encontrados: {set(estrangeirismos)}")
print(f"Gírias encontradas: {set(girias)}")

Exemplos de stopwords em português: ['dá', 'foste', 'coisa', 'já', 'atrás', 'todos', 'logo', 'possível', 'dão', 'doze']
Número total de palavras: 12009
Número total de palavras únicas: 3381
Riqueza lexical: 0.2815
Palavras mais comuns: [('de', 436), ('que', 430), ('a', 412), ('e', 358), ('o', 329), ('não', 175), ('é', 173), ('do', 151), ('da', 143), ('se', 124)]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Número de palavras sem stopwords: 5637
Número de palavras únicas sem stopwords: 3084
Riqueza lexical sem stopwords: 0.5471
Palavras mais comuns (sem stopwords): [('pessoa', 36), ('partido', 35), ('há', 34), ('chega', 31), ('and', 28), ('fernando', 28), ('by', 24), ('mundo', 21), ('to', 19), ('gostargostar', 19)]
------------------------------------------------------------------------------------------------------------------------

#### Palavras novas no teste

In [28]:
# 1. Tokenização simples por espaço
tokens_train_espaco = set(train.lower().split())
tokens_test_espaco = set(test.lower().split())

# 2. Tokenização com expressões regulares
tokens_train_regex = set(re.findall(r'\w+', train.lower()))
tokens_test_regex = set(re.findall(r'\w+', test.lower()))

# 3. Tokenização com spacy
tokens_train_spacy = {token.text.lower() for token in nlp(train)}
tokens_test_spacy = {token.text.lower() for token in nlp(test)}

# Encontrar palavras únicas no teste
def palavras_novas(train, test):
    return test - train

resultados = {
    "Espaços em branco": palavras_novas(tokens_train_espaco, tokens_test_espaco),
    "Expressões regulares": palavras_novas(tokens_train_regex, tokens_test_regex),
    "spaCy": palavras_novas(tokens_train_spacy, tokens_test_spacy),
}

for metodo, palavras in resultados.items():
    print(f"\nMétodo: {metodo}")
    print(f"Número de palavras novas: {len(palavras)}")
    print(f"Palavras novas: {palavras}")


Método: Espaços em branco
Número de palavras novas: 360
Palavras novas: {'decisão', 'animalescas', 'tamanho', 'ladrõezecos', 'força', 'pipoca', 'dão', 'cleptomania,', 'votar', 'proveitos', 'sedentos', 'transparentes', 'olhos!', 'contar', 'economia)', 'governante,', 'gostam', 'unido', 'fazer!', 'eleger.', 'furiosas.', 'parolice!', 'suficiente', 'peso', 'vacina.', 'suíços),', 'ferozes', 'diminuir', 'levanta', 'artífice', 'cadeiras', 'mole', 'bons,', 'necessita', 'direita;', 'liderança,', 'aprendem', 'cinquenta', 'cor', 'rirmos', 'vontade,', 'using', 'económicas', 'sexo', 'humana.', 'sexuais,etc,etc,etc', 'juízo', 'elon', 'cotrim', 'ganham', 'costuma', 'parvalhões', 'tutti', 'excelente,', '(muita', 'condenar', 'estado,', 'tvs', 'voto?', 'costumam', 'países).', 'eleitoral', 'ideológico', 'perpetuá-las.', 'governantes', 'brando', 'folgam', 'parâmetros', 'argumento', 'duro,', 'ferramenta', 'chamadas', 'deveriam', 'improfícuo.', 'américa', 'lado,', '“das', '80', 'coelho,', '“que', 'lebres', 

## Tokenizer

In [29]:
class BPE:
    """
    A Byte Pair Encoding (BPE) tokenizer implementation.

    This class tokenizes input text using the BPE algorithm, which iteratively merges the most
    frequent character pairs into new tokens until a specified vocabulary size is reached.

    Attributes:
    -----------
    text : The input text to be tokenized (str)
    vocab_size : The desired vocabulary size for the tokenized output (int)
    alphabet: A list of unique characters in the input text (list)
    vocab: A dictionary mapping character pairs to their merged tokens (dict)
    splits: A dictionary mapping words to their character splits (dict)
    """

    def __init__(self, text_string, vocab_size):
        """
        Initializes the BPE tokenizer.

        Parameters:
            text_string: The input text to be tokenized (str)
            vocab_size: The desired size of the vocabulary (int)
        """

        self.text = self.pre_tokenize_str(text_string)  # List of tokens
        self.vocab_size = vocab_size
        self.alphabet = []
        self.vocab = {}
        self.splits = {word: list(word) for word in self.text}

    def pre_tokenize_str(self, text):
        """
        Pre-tokenizes the input text by splitting words and adding a special character before each new word.

        Parameters:
            text: The input text (str)

        Returns:
            pre_tokenized_text: A list of pre-tokenized words (list)
        """

        tokens = text.split()
        pre_tokenized_text = [tokens[0].lower()] + ["Ġ" + word.lower() if word.isalnum() else word for word in tokens[1:]]
        return pre_tokenized_text

    def words_freq(self):
        """
        Computes the frequency of words in the text.

        Returns:
            A dictionary where keys are words and values are their respective frequencies (dict)
        """

        freq = Counter(self.text)
        return dict(sorted(freq.items(), key=lambda x: x[1], reverse=True))

    def tokenizer(self):
        """"
        Extracts the alphabet from the given text.

        Returns:
            A list of unique characters present in the text (list)
        """

        for word in self.words_freq().keys():
            for letter in word:
                if letter not in self.alphabet:
                    self.alphabet.append(letter)
        return self.alphabet

    def compute_pair_freqs(self):
        """
        Computes the frequency of character pairs in the text.

        Returns:
            A dictionary where keys are character pairs and values are their frequencies (dict)
        """

        pair_freqs = defaultdict(int)
        for word, freq in self.words_freq().items():
            split = self.splits[word]
            if len(split) == 1:
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] += freq
        return pair_freqs

    def compute_best_pair(self, pair_freqs):
        """
        Identifies the most frequent character pair.

        Parameters:
            pair_freqs: A dictionary of character pairs and their frequencies (dict)

        Returns:
            The most frequent character pair and its frequency (tuple)
        """

        return max(pair_freqs.items(), key=lambda x: x[1], default=(None, None))

    def merge_pair(self, a, b):
        """
        Merges a given character pair in all words.

        Parameters:
            a: The first character of the pair (str)
            b: The second character of the pair (str)
        """

        for word in self.words_freq():
            split = self.splits[word]
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    split = split[:i] + [a + b] + split[i + 2:]
                else:
                    i += 1
            self.splits[word] = split

    def train_tokenize(self):
        """
        Trains the BPE model by iteratively merging the most frequent character pairs until
        the vocabulary reaches the specified size.
        """

        self.vocab = {}
        while len(self.vocab) < self.vocab_size:
            pair_freqs = self.compute_pair_freqs()
            best_pair, max_freq = self.compute_best_pair(pair_freqs)
            if best_pair is None:
                break
            self.merge_pair(*best_pair)
            self.vocab[best_pair] = best_pair[0] + best_pair[1]

    def bpe_tokenization(self, text):
        """
        Tokenizes input text using the trained BPE model.

        Parameters:
            text: The input text to tokenize (str)

        Returns:
            A list of tokenized words (list)
        """

        text = self.pre_tokenize_str(text)
        splits = [[l for l in word] for word in text]

        for pair, merge in self.vocab.items():
            for idx, split in enumerate(splits):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2:]
                    else:
                        i += 1
                    splits[idx] = split
        tokens = sum(splits, [])

        return tokens, text

In [30]:
# Run BPE with an string

bpe = BPE(train, vocab_size=1000)
bpe.train_tokenize()
tokens, text = bpe.bpe_tokenization("Hello world, this is a BPE tokenizer test.")

print("Original text:", text)
print("Tokens:", tokens)

Original text: ['hello', 'world,', 'Ġthis', 'Ġis', 'Ġa', 'Ġbpe', 'Ġtokenizer', 'test.']
Tokens: ['h', 'el', 'lo', 'w', 'or', 'l', 'd', ',', 'Ġt', 'h', 'is', 'Ġis', 'Ġa', 'Ġb', 'pe', 'Ġto', 'k', 'en', 'iz', 'er', 'tes', 't', '.']


In [31]:
# Run BPE with the test set

tokens, text = bpe.bpe_tokenization(test)

print("Original text:", text)
print("Tokens:", tokens)

Original text: ['ões.', 'Assim,', 'Ġsempre', 'Ġque', 'Ġhouve', 'prevaricação,', 'Ġambos', 'Ġos', 'lados,', 'Ġconsoante', 'Ġa', 'Ġcor', 'Ġpolítica', 'governante,', 'Ġdesculpabilizaram', 'Ġos', 'Ġseus', 'Ġpolíticos', 'Ġem', 'Ġvez', 'Ġde', 'Ġos', 'Ġcastigar', 'Ġseveramente', 'Ġe', 'Ġexigir', 'Ġque', 'Ġfossem', 'Ġbanidos', 'Ġda', 'política.', 'Sim,', 'sim,', 'Ġa', 'Ġculpa', 'Ġé', 'Ġmesmo', 'Ġdo', 'Ġpovo', 'Ġbrando', 'Ġque', 'Ġassistiu', 'Ġa', 'Ġesta', 'Ġpouca', 'Ġvergonha', 'Ġao', 'Ġlongo', 'Ġdos', 'Ġtempos', 'Ġe', 'Ġsimplesmente', 'Ġencolheu', 'Ġos', 'ombros.', 'Ġquando', 'Ġé', 'Ġque', 'Ġaprendem', 'Ġque', 'Ġo', 'Ġreal', 'Ġpoder', 'Ġestá', 'Ġna', 'Ġforça', 'Ġdo', 'Ġpovo', 'Ġunido', 'Ġpara', 'Ġfazer', 'Ġvaler', 'Ġos', 'Ġvossos', 'Ġdireitos', 'Ġe', 'Ġresponsabilizar', 'Ġquem', 'Ġtem', 'Ġa', 'Ġobrigação', 'Ġde', 'Ġhonrar', 'Ġo', 'Ġvosso', 'voto?', 'Ġnão', 'Ġimporta', 'Ġse', 'Ġé', 'Ġde', 'Ġesquerda', 'Ġou', 'Ġde', 'direita;', 'Ġquando', 'Ġse', 'Ġdesculpa', 'Ġem', 'Ġvez', 'Ġde', 'Ġcondenar', '

## Comparação

In [34]:
from transformers import AutoTokenizer

# Tokenizadores pré-treinados
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
wordpiece_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
unigram_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [36]:
#bpe = BPE(train, vocab_size=500)
bpe_tokens, text_string = bpe.bpe_tokenization(test)
gpt2_tokens = gpt2_tokenizer.tokenize(test)
wordpiece_tokens = wordpiece_tokenizer.tokenize(test)
unigram_tokens = unigram_tokenizer.tokenize(test)

# Texto com palavras fora do vocabulário
fora_vocab = "oftalmologista fisioterapeuticamente desoxirribonucleoproteína"

#bpe = BPE(train, vocab_size=500)
bpe_oov_tokens, text_oov_string = bpe.bpe_tokenization(fora_vocab)
gpt2_oov_tokens = gpt2_tokenizer.tokenize(fora_vocab)
wordpiece_oov_tokens = wordpiece_tokenizer.tokenize(fora_vocab)
unigram_oov_tokens = unigram_tokenizer.tokenize(fora_vocab)

# comparação de número de tokens
print("\n=== Número de Tokens ===")
print(f"BPE (GPT-2): {len(gpt2_tokens)} tokens -> {gpt2_tokens}")
print(f"WordPiece (BERT): {len(wordpiece_tokens)} tokens -> {wordpiece_tokens}")
print(f"Unigram (XLM-R): {len(unigram_tokens)} tokens -> {unigram_tokens}")
print(f"BPE: {len(bpe_tokens)} tokens -> {bpe_tokens}")

# Comparação de segmentação de palavras
print("\n=== Segmentação das Palavras ===")
print("BPE (GPT2):", " | ".join(gpt2_tokens))
print("WordPiece:", " | ".join(wordpiece_tokens))
print("Unigram:", " | ".join(unigram_tokens))
print("BPE:", " | ".join(bpe_tokens))

# Comparação de palavras fora do vocabulário
print("\n=== Palavras Fora do Vocabulário ===")
print(f"BPE (GPT-2): {gpt2_oov_tokens}")
print(f"WordPiece (BERT): {wordpiece_oov_tokens}")
print(f"Unigram (XLM-R): {unigram_oov_tokens}")
print(f"BPE: {bpe_oov_tokens}")


=== Número de Tokens ===
BPE (GPT-2): 2725 tokens -> ['Ã', 'µ', 'es', '.', 'ĠAss', 'im', ',', 'Ġsem', 'pre', 'Ġque', 'Ġh', 'ou', 've', 'Ġpre', 'var', 'ica', 'Ã§', 'Ã£o', ',', 'Ġamb', 'os', 'Ġos', 'Ġlad', 'os', ',', 'Ġcon', 'so', 'ante', 'Ġa', 'Ġcor', 'Ġpol', 'ÃŃ', 't', 'ica', 'Ġgovern', 'ante', ',', 'Ġdesc', 'ulp', 'abil', 'iz', 'aram', 'Ġos', 'Ġse', 'us', 'Ġpol', 'ÃŃ', 'tic', 'os', 'Ġem', 'Ġve', 'z', 'Ġde', 'Ġos', 'Ġcast', 'ig', 'ar', 'Ġsever', 'ament', 'e', 'Ġe', 'Ġex', 'ig', 'ir', 'Ġque', 'Ġf', 'os', 'sem', 'Ġban', 'id', 'os', 'Ġda', 'Ġpol', 'ÃŃ', 't', 'ica', '.', 'ĠSim', ',', 'Ġsim', ',', 'Ġa', 'Ġcul', 'pa', 'ĠÃ©', 'Ġmes', 'mo', 'Ġdo', 'Ġp', 'ovo', 'Ġbrand', 'o', 'Ġque', 'Ġassist', 'iu', 'Ġa', 'Ġest', 'a', 'Ġp', 'ou', 'ca', 'Ġver', 'gon', 'ha', 'Ġa', 'o', 'Ġlong', 'o', 'Ġdos', 'Ġtem', 'pos', 'Ġe', 'Ġsim', 'ples', 'ment', 'e', 'Ġenc', 'ol', 'he', 'u', 'Ġos', 'Ġo', 'mb', 'ros', '.', 'ĠQu', 'ando', 'ĠÃ©', 'Ġque', 'Ġap', 'rend', 'em', 'Ġque', 'Ġo', 'Ġreal', 'Ġp', 'oder', 'Ġest', 'Ã¡',